<a href="https://colab.research.google.com/github/lweislo/CNResults/blob/master/letour_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This page lets you scrape results directly from any LeTour.fr-formatted race.
# Please follow these instructions!
* Shift-enter each cell or click play button by hovering over the [  ] above the start() cell to load the code
* After running the cells above start(), you can just run the start() cell alone multiple times.
* Enter the full URL including the stage number and http:// in at the prompt (see example)
* The file will directly download as aso_results.csv to your default downloads folder
* Open CSV file in TextMate or other text editor, change commas to tabs then paste into Excel for best results with accents.

* email laura.weislo@futurenet.com if something breaks!



In [0]:
import requests
import html5lib
import pandas as pd
from bs4 import BeautifulSoup
import os
import json
import numpy as np
import datetime
from google.colab import files

In [0]:
# 2019 Tour de France bib macro
tour_19 = {1:"Geraint Thomas (GBr) Team Ineos",
2:"Egan Bernal (Col) Team Ineos",
3:"Jonathan Castroviejo (Spa) Team Ineos",
4:"Michal Kwiatkowski (Pol) Team Ineos",
5:"Gianni Moscon (Ita) Team Ineos",
6:"Wout Poels (Ned) Team Ineos",
7:"Luke Rowe (GBr) Team Ineos",
8:"Dylan van Baarle (Ned) Team Ineos",
11:"Peter Sagan (Svk) Bora-Hansgrohe",
12:"Emanuel Buchmann (Ger) Bora-Hansgrohe",
13:"Marcus Burghardt (Ger) Bora-Hansgrohe",
14:"Patrick Konrad (Aut) Bora-Hansgrohe",
15:"Gregor Mühlberger (Aut) Bora-Hansgrohe",
16:"Daniel Oss (Ita) Bora-Hansgrohe",
17:"Lukas Pöstlberger (Aut) Bora-Hansgrohe",
18:"Maximilian Schachmann (Ger) Bora-Hansgrohe",
21:"Julian Alaphilippe (Fra) Deceuninck-QuickStep",
22:"Kasper Asgreen (Den) Deceuninck-QuickStep",
23:"Dries Devenyns (Bel) Deceuninck-QuickStep",
24:"Yves Lampaert (Bel) Deceuninck-QuickStep",
25:"Enric Mas (Spa) Deceuninck-QuickStep",
26:"Michael Mørkøv (Den) Deceuninck-QuickStep",
27:"Maximiliano Richeze (Arg) Deceuninck-QuickStep",
28:"Elia Viviani (Ita) Deceuninck-QuickStep",
31:"Romain Bardet (Fra) AG2R La Mondiale",
32:"Mikael Cherel (Fra) AG2R La Mondiale",
33:"Benoit Cosnefroy (Fra) AG2R La Mondiale",
34:"Mathias Frank (Swi) AG2R La Mondiale",
35:"Tony Gallopin (Fra) AG2R La Mondiale",
36:"Alexis Gougeard (Fra) AG2R La Mondiale",
37:"Oliver Naesen (Bel) AG2R La Mondiale",
38:"Alexis Vuillermoz (Fra) AG2R La Mondiale",
41:"Vincenzo Nibali (Ita) Bahrain-Merida",
42:"Damiano Caruso (Ita) Bahrain-Merida",
43:"Sonny Colbrelli (Ita) Bahrain-Merida",
44:"Rohan Dennis (Aus) Bahrain-Merida",
45:"Ivan Garcia (Spa) Bahrain-Merida",
46:"Matej Mohoric (Slo) Bahrain-Merida",
47:"Dylan Teuns (Bel) Bahrain-Merida",
48:"Jan Tratnik (Slo) Bahrain-Merida",
51:"Thibaut Pinot (Fra) Groupama-FDJ",
52:"William Bonnet (Fra) Groupama-FDJ",
53:"David Gaudu (Fra) Groupama-FDJ",
54:"Stefan Küng (Swi) Groupama-FDJ",
55:"Matthieu Ladagnous (Fra) Groupama-FDJ",
56:"Rudy Molard (Fra) Groupama-FDJ",
57:"Sébastien Reichenbach (Swi) Groupama-FDJ",
58:"Anthony Roux (Fra) Groupama-FDJ",
61:"Nairo Quintana (Col) Movistar Team",
62:"Alejandro Valverde (Spa) Movistar Team",
63:"Andrey Amador (CRc) Movistar Team",
64:"Imanol Erviti (Spa) Movistar Team",
65:"Mikel Landa (Spa) Movistar Team",
66:"Nelson Oliveira (Por) Movistar Team",
67:"Marc Soler (Spa) Movistar Team",
68:"Carlos Verona (Spa) Movistar Team",
71:"Jakob Fuglsang (Den) Astana Pro Team",
72:"Pello Bilbao (Spa) Astana Pro Team",
73:"Omar Fraile (Spa) Astana Pro Team",
74:"Hugo Houle (Can) Astana Pro Team",
75:"Gorka Izagirre Insausti (Spa) Astana Pro Team",
76:"Alexey Lutsenko (Kaz) Astana Pro Team",
77:"Magnus Cort Nielsen (Den) Astana Pro Team",
78:"Luis León Sanchez (Spa) Astana Pro Team",
81:"Steven Kruijswijk (Ned) Team Jumbo-Visma",
82:"George Bennett (NZl) Team Jumbo-Visma",
83:"Laurens De Plus (Bel) Team Jumbo-Visma",
84:"Dylan Groenewegen (Ned) Team Jumbo-Visma",
85:"Amund Grøndahl Jansen (Nor) Team Jumbo-Visma",
86:"Tony Martin (Ger) Team Jumbo-Visma",
87:"Mike Teunissen (Ned) Team Jumbo-Visma",
88:"Wout Van Aert (Bel) Team Jumbo-Visma",
91:"Rigoberto Uran (Col) EF Education First",
92:"Alberto Bettiol (Ita) EF Education First",
93:"Simon Clarke (Aus) EF Education First",
94:"Tanel Kangert (Est) EF Education First",
95:"Sebastian Langeveld (Ned) EF Education First",
96:"Tom Scully (NZl) EF Education First",
97:"Tejay Van Garderen (USA) EF Education First",
98:"Michael Woods (Can) EF Education First",
101:"Adam Yates (GBr) Mitchelton-Scott",
102:"Luke Durbridge (Aus) Mitchelton-Scott",
103:"Jack Haig (Aus) Mitchelton-Scott",
104:"Michael Hepburn (Aus) Mitchelton-Scott",
105:"Daryl Impey (RSA) Mitchelton-Scott",
106:"Christopher Juul Jensen (Den) Mitchelton-Scott",
107:"Matteo Trentin (Ita) Mitchelton-Scott",
108:"Simon Yates (GBr) Mitchelton-Scott",
111:"Greg Van Avermaet (Bel) CCC Team",
112:"Patrick Bevin (NZl) CCC Team",
113:"Alessandro De Marchi (Ita) CCC Team",
114:"Simon Geschke (Ger) CCC Team",
115:"Serge Pauwels (Bel) CCC Team",
116:"Joseph Rosskopf (USA) CCC Team",
117:"Michael Schär (Swi) CCC Team",
118:"Lukasz Wisniowski (Pol) CCC Team",
121:"Daniel Martin (Irl) UAE Team Emirates",
122:"Fabio Aru (Ita) UAE Team Emirates",
123:"Sven Erik Bystrøm (Nor) UAE Team Emirates",
124:"Rui Costa (Por) UAE Team Emirates",
125:"Sergio Henao (Col) UAE Team Emirates",
126:"Alexander Kristoff (Nor) UAE Team Emirates",
127:"Vegard Stake Laengen (Nor) UAE Team Emirates",
128:"Jasper Philipsen (Bel) UAE Team Emirates",
131:"Richie Porte (Aus) Trek-Segafredo",
132:"Julien Bernard (Fra) Trek-Segafredo",
133:"Giulio Ciccone (Ita) Trek-Segafredo",
134:"Koen de Kort (Ned) Trek-Segafredo",
135:"Fabio Felline (Ita) Trek-Segafredo",
136:"Bauke Mollema (Ned) Trek-Segafredo",
137:"Toms Skujins (Lat) Trek-Segafredo",
138:"Jasper Stuyven (Bel) Trek-Segafredo",
141:"Michael Matthews (Aus) Team Sunweb",
142:"Nikias Arndt (Ger) Team Sunweb",
143:"Cees Bol (Ned) Team Sunweb",
144:"Chad Haga (USA) Team Sunweb",
145:"Lennard Kämna (Ger) Team Sunweb",
146:"Wilco Kelderman (Ned) Team Sunweb",
147:"Søren Kragh Andersen (Den) Team Sunweb",
148:"Nicolas Roche (Irl) Team Sunweb",
151:"Christophe Laporte (Fra) Cofidis Solutions Credits",
152:"Natnael Berhane (Eri) Cofidis Solutions Credits",
153:"Nicolas Edet (Fra) Cofidis Solutions Credits",
154:"Jesus Herrada (Spa) Cofidis Solutions Credits",
155:"Anthony Perez (Fra) Cofidis Solutions Credits",
156:"Pierre Luc Perichon (Fra) Cofidis Solutions Credits",
157:"Stéphane Rossetto (Fra) Cofidis Solutions Credits",
158:"Julien Simon (Fra) Cofidis Solutions Credits",
161:"Caleb Ewan (Aus) Lotto Soudal",
162:"Tiesj Benoot (Bel) Lotto Soudal",
163:"Jasper De Buyst (Bel) Lotto Soudal",
164:"Thomas De Gendt (Bel) Lotto Soudal",
165:"Jens Keukeleire (Bel) Lotto Soudal",
166:"Roger Kluge (Ger) Lotto Soudal",
167:"Maxime Monfort (Bel) Lotto Soudal",
168:"Tim Wellens (Bel) Lotto Soudal",
171:"Lilian Calmejane (Fra) Total Direct Energie",
172:"Niccolò Bonifazio (Ita) Total Direct Energie",
173:"Fabien Grellier (Fra) Total Direct Energie",
174:"Paul Ourselin (Fra) Total Direct Energie",
175:"Romain Sicard (Fra) Total Direct Energie",
176:"Rein Taaramäe (Est) Total Direct Energie",
177:"Niki Terpstra (Ned) Total Direct Energie",
178:"Anthony Turgis (Fra) Total Direct Energie",
181:"Ilnur Zakarin (Rus) Katusha-Alpecin",
182:"Jens Debusschere (Bel) Katusha-Alpecin",
183:"Alex Dowsett (GBr) Katusha-Alpecin",
184:"José Gonçalves (Por) Katusha-Alpecin",
185:"Marco Haller (Aut) Katusha-Alpecin",
186:"Nils Politt (Ger) Katusha-Alpecin",
187:"Mads Würtz Schmidt (Den) Katusha-Alpecin",
188:"Rick Zabel (Ger) Katusha-Alpecin",
191:"Guillaume Martin (Fra) Wanty-Gobert",
192:"Frederik Backaert (Bel) Wanty-Gobert",
193:"Aime De Gendt (Bel) Wanty-Gobert",
194:"Odd Christian Eiking (Nor) Wanty-Gobert",
195:"Xandro Meurisse (Bel) Wanty-Gobert",
196:"Yoann Offredo (Fra) Wanty-Gobert",
197:"Andrea Pasqualon (Ita) Wanty-Gobert",
198:"Kevin Van Melsen (Bel) Wanty-Gobert",
198:"Kevin Van Melsen (Bel) Wanty-Gobert",
201:"Edvald Boasson Hagen (Nor) Dimension Data",
202:"Lars Bak (Den) Dimension Data",
203:"Stephen Cummings (GBr) Dimension Data",
204:"Reinardt Janse Van Rensburg (RSA) Dimension Data",
205:"Ben King (USA) Dimension Data",
206:"Roman Kreuziger (Cze) Dimension Data",
207:"Giacomo Nizzolo (Ita) Dimension Data",
208:"Michael Valgren Andersen (Den) Dimension Data",
211:"Warren Barguil (Fra) Arkéa Samsic",
212:"Maxime Bouet (Fra) Arkéa Samsic",
213:"Anthony Delaplace (Fra) Arkéa Samsic",
214:"Elie Gesbert (Fra) Arkéa Samsic",
215:"André Greipel (Ger) Arkéa Samsic",
216:"Kevin Ledanois (Fra) Arkéa Samsic",
217:"Amael Moinard (Fra) Arkéa Samsic",
218:"Florian Vachon (Fra) Arkéa Samsic"}
# ASO code for Tour classifications in page source
tab_dict = {'ite':'Stage',
'ipe':'Points',
'ime':'Mountains',
'ije':'Young riders',
'ice':'Combativity',
'ete':'Teams',
'itg':'General Classification',
'ipg':'Points Classification',
'img':'Mountains Classification',
'ijg':'Young Riders Classification',
'icg':'Combativity Classification',
'etg':'Teams Classification'}

order_list = ['ite', 'ipe', 'ime', 'ije', 'ice', 'ete', 'itg', 'ipg', 'img', 'icg','ijg', 'etg']
points_stg = ['ipe', 'ime']

In [0]:
# ALL THE CODE YOU NEED

def output_file(header_list, table_list):
    outfile = 'aso_results.csv'
    with open(outfile, 'w') as file:
        for item in range(0, len(header_list)):
            try:
                file.write(f'\n{header_list[item]}\n')
                file.write(table_list[item].to_csv(header=False, index=False, encoding='Latin-1'))
            except:
                pass
    files.download(outfile)
# Scrape the page for stage and extracts a list of raw URLs coded with the classification code

def scrape_urls(stage_url):
    base_url = stage_url.split('/')[2]
    links_list = [] #Temporarily store the list of unparsed URLs
    url_dict = {} #Dictionary with result type code and full URL
    print(f"Getting result links for {stage_url}")

    page = requests.get(stage_url)

    if page.status_code == 200:
        content = page.content
        soup = BeautifulSoup(content, "html5lib")

    #Pull out a specific block of code with two sets of coded URLs from the soup. YMMV if ASO change the format!
        try:
            all_links = soup.find(class_="ranking__tabs") #This is from a UL tag above the table on the page
            links = all_links.find_all(class_="tabs__link") #There are two items in the list, stage and GC
            
            for item in links:
                urls = item.get('data-ajax-stack')
                links_list.append(urls)

        except ElementDoesNotExist as e:
            print(f"That does not appear to be a valid results URL. {e}")
# Parse out the URLs from the list 'data-ajax-stack' attribute


    for item in links_list:
          #clean up the code into a useable URL
        urls = item.replace('\'', '"')
        myurl = urls.replace('\/', '/')
        myurls = json.loads(myurl)
        for key, value in myurls.items():
            url_dict[key] = f"https://{base_url}{value}"
            
    for item in order_list:   # IMPORTANT: We want to process these in the order of the universal result order list 
        try:
            print(f"Scraping results from {tab_dict[item], url_dict[item]}")
            scrape_results(item, url_dict[item], stage_url)
        except KeyError:
            print(f"No results for {tab_dict[item]}")
            output_tables.append("")
            pass

def scrape_results(item, url, stage_url):
    print(f"Getting results for {item}, {url}")
    out_df = pd.DataFrame()

    try:
        page = requests.get(url).content
        soup = BeautifulSoup(page, "html5lib")
#Treat sprint and mountains stage results differently, they have multiple tables per page
        if item == 'ipe' or item == 'ime':
            if item == 'ipe':
                name = "Sprint"
            else:
                name = "Mountain"
            print(f"Found a {name}")
            mydiv = soup.find(class_="tabs__content")
            labels = mydiv.find_all(class_="rankingTables__caption")
            tables = mydiv.find_all(class_="rankingTable")

            for item in labels:
                try:
                    this_label = item.text.title()
                    this_label = this_label.replace(" Km", "").replace(" - "," km. ")
                    this_label = (f"{name} {labels.index(item)+1} - {this_label}")
                    print(this_label)
                    output_labels.append(this_label)
                except ValueError:
                    output_labels.append("")
                    pass

            for item in tables:
                out_df = pd.DataFrame()
                try:
                    table = pd.read_html(str(item), index_col=None, header=None)
                    table = table[0]
                    out_df['Place'] = table['Rank']
                    out_df['Bib'] = table['Rider No.'].map(tour_19).fillna(table['Rider No.'])
                    out_df['Points'] = table['Points'].str.replace('PTS', '')
                    output_tables.append(out_df)
                except ValueError:
                    pass        
        else:
            # Fish the tables out of the soup
            res_table = pd.read_html(page)
            df = res_table[0]
            # Get the heading for output
            label = tab_dict[item]
            output_labels.append(label)
            # Get the table for output, formatting points and times differently
            if item in ['ite', 'ije', 'itg', 'ijg']:
                out_df['Place'] = df['Rank'].fillna('')
                out_df['Bib'] = df['Rider No.'].map(tour_19).fillna(df['Rider No.'])
                out_df['Result'] = df['Times'].str.replace('h ', ':').str.replace('\'\'', '').str.replace('\' ',':')
#                 For the stage results, go to the withdrawals function and append any for this stage to the end
                if item == 'ite':
                    try:
                        df = withdrawals(stage_url)
                        print(df)
                        out_df = out_df.append(df)
                    except:
                        pass
            elif item in ['ipg', 'img']:
                out_df['Place'] = df['Rank'].fillna('')
                out_df['Bib'] = df['Rider No.'].map(tour_19).fillna(df['Rider No.'])
                out_df['Result'] = df['Points'].str.replace(' PTS', '')
            elif item == 'ice':
                out_df['Place'] = df['Rank'].fillna('')
                out_df['Bib'] = df['Rider No.'].map(tour_19).fillna(df['Rider No.'])
                out_df['Result'] = ''
            elif item in ['ete', 'etg']:
                out_df['Place'] = df['Rank'].fillna('')
                out_df['Bib'] = df['Team'].fillna('')
                out_df['Result'] = df['Times'].str.replace('h ', ':').str.replace('\'\'', '').str.replace('\' ',':')
            
            output_tables.append(out_df)


    except ValueError:
        print("No table found for " + item)  
        
        
def withdrawals(stage_url):

    base_url = stage_url.split('/')[2]
    stage_id = stage_url.split('/')[-1].title().replace('-', ' ') # Results in Stage X, the text inside div tag for that stage on withdrawals page
    wd_url = f"https://{base_url}/en/withdrawal" #YMMV IF ASO CHANGE THEIR SITE STRUCTURE
    print(f'Getting withdrawals for stage {stage_id}')

    page = requests.get(wd_url)
    out_df = pd.DataFrame() # Pick out the columns we want and put in a new dataframe to append    
    if page.status_code == 200:
        content = page.content
        soup = BeautifulSoup(content, "html5lib")
        divs = soup.find_all('div', class_="rankingTables__item") # Parent of each stage's withdrawal list
        wds = soup.find('div', text=stage_id).parent # Pull out only the table for the stage we want
        wdt = wds.find('table', class_="rankingTable rtable") # Pull the table out of the div
        wdt = pd.read_html(str(wdt))[0] # Format the table as a dataframe
        out_df['Place'] = wdt['Withdraw'].str.replace('withdrawal','DNF')
        out_df['Bib'] = wdt['Bib'].map(tour_19).fillna(wdt['Bib'])
        out_df['Result'] = ''
    print(out_df)
    
    stage_id = ""
    return out_df # Returns this to the function line 95

def start():

    stage_url = input("Enter the URL to LeTour results including stage id, eg https://www.criterium-du-dauphine.fr/en/rankings/stage-5 : ")
    scrape_urls(stage_url)
    output_file(output_labels, output_tables)

## THIS STARTS THE PROGRAM RUNNING
* You only need to run all the cells above once before clicking the cell below to start extracting results

In [41]:
output_labels = []
output_tables = []
start()
# https://www.criterium-du-dauphine.fr/en/rankings/stage-2

Enter the URL to LeTour results including stage id, eg https://www.criterium-du-dauphine.fr/en/rankings/stage-5 : https://www.letour.fr/en/rankings/stage-20
Getting result links for https://www.letour.fr/en/rankings/stage-20
Scraping results from ('Stage', 'https://www.letour.fr/en/ajax/ranking/20/ite/75b54491d06887ea121cfe2f484a725d/none')
Getting results for ite, https://www.letour.fr/en/ajax/ranking/20/ite/75b54491d06887ea121cfe2f484a725d/none
Getting withdrawals for stage Stage 20
Scraping results from ('Points', 'https://www.letour.fr/en/ajax/ranking/20/ipe/cb373b0e93432e51a13f1ed55230ec2e/none')
Getting results for ipe, https://www.letour.fr/en/ajax/ranking/20/ipe/cb373b0e93432e51a13f1ed55230ec2e/none
Found a Sprint
Sprint 1 - Espelette km. 31
Scraping results from ('Mountains', 'https://www.letour.fr/en/ajax/ranking/20/ime/c9853a38361aec30f28e11e3d97b3b7b/none')
Getting results for ime, https://www.letour.fr/en/ajax/ranking/20/ime/c9853a38361aec30f28e11e3d97b3b7b/none
Found a Mo